## Load the Motion Data

Data is in x and y coordinates for each pixel. Each sample will be an array of 10 (frames) x 40 x 40 (capture window) x 2 (x and y) dimensions. 

In [1]:
import numpy as np
import os
import matplotlib
matplotlib.use('TkAgg') # For displaying animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from helper_functions import *
from normalize_data import *
from integral_try import *
from numpy import array
from sklearn import preprocessing as pp

normalization function imported
Integral functions imported


In [2]:
data_dir = 'sonic_pi_face/data/'

# Get list of data files
data_files = get_data_files(data_dir)

# Load data into a dictionary
# Note: Checks for incomplete data
data = get_gesture_data(data_files)

# Get number of `open-close` samples
len(data['open-close'])

122

## Normalize data sample and get integral image for feature detection

In [9]:
# Normalize data across data samples and frames (x and y will be normalized independently)
data = normalize_data(data)
print(np.shape(data['open-close']))

# Load first data sample from `open-close`
sample = data['open-close'][0]
print(np.shape(sample))

# calculate the integral image
iimage = i_image(sample[4])

# calculate the integral for the first quarter from the integral image
try_integral = get_integral(iimage,0,0,20,20)


print('Example integral of the 1st quarter of one frame: ' , try_integral)

(122, 10, 40, 40, 2)
(10, 40, 40, 2)
Example integral of the 1st quarter of one frame:  [ 294.85297279  346.1337839 ]


## Visualize horizontal motion across frames

In [11]:
sample = data['open-close'][121] # Fifth sample
frame = sample[5][:,:,0] # Middle frame
# Note: Second argument takes 0 for `x`, 1 for `y` coordinate
display_frames(sample,0) 



KeyboardInterrupt: 